In [1]:
import copy 

input = open('input.txt').read().strip().split('\n')
input = [list(line) for line in input]

# add grid of . around the input
for line in input:
    line.insert(0, '.')
    line.append('.')

input.insert(0, ['.']*len(input[0]))
input.append(['.']*len(input[0]))

def check_surrounding(area, pos):
    for r,p in area:
        loc = [input[r-1][p], input[r+1][p], input[r][p-1], input[r][p+1]]
        if pos in loc:
            return True
    return False


def return_adjacent(area,label):
    region = copy.deepcopy(area)
    found_neighbours = set(area)
    for el in region:
        # check neighbours
        if input[el[0]-1][el[1]] ==label:
            found_neighbours.add((el[0]-1,el[1]))
            if (el[0]-1,el[1]) not in region:
                region.append((el[0]-1,el[1]))
        if input[el[0]+1][el[1]] == label:
            found_neighbours.add((el[0]+1,el[1]))
            if (el[0]+1,el[1]) not in region:
                region.append((el[0]+1,el[1]))
        if input[el[0]][el[1]-1] == label:
            found_neighbours.add((el[0],el[1]-1))
            if (el[0],el[1]-1) not in region:
                region.append((el[0],el[1]-1))
        if input[el[0]][el[1]+1] == label:
            found_neighbours.add((el[0],el[1]+1))
            if (el[0],el[1]+1) not in region:
                region.append((el[0],el[1]+1))
    return found_neighbours


already_checked = set()

area = []
current = None

neig_areas = []

for r in range(len(input)):
    for p in range(len(input[0])):
        if not (r,p) in already_checked and not input[r][p] == '.':
            if input[r][p] == current:
                if check_surrounding(area,(r,p)):
                    area.append((r,p))
                    already_checked.add((r,p))
                else:
                    neig_area = return_adjacent(area,current)
                    neig_areas.append({'label':current, 'pos':neig_area})
                    for neig in neig_area:
                        already_checked.add(neig)
                    current = input[r][p]
                    area = [(r,p)]
                    already_checked.add((r,p)) 

        
            if input[r][p] != current:
                if current is not None:
                    neig_area = return_adjacent(area,current)
                    neig_areas.append({'label':current, 'pos':neig_area})
                    for neig in neig_area:
                        already_checked.add(neig)
                    current = input[r][p]
                    area = [(r,p)]
                    already_checked.add((r,p)) 
                else:
                    current = input[r][p]
                    area = [(r,p)]
                    already_checked.add((r,p))

neig_area = return_adjacent(area,current)
neig_areas.append({'label':current, 'pos':neig_area})

# remove duplicates
final_neig_areas = []
for neig_area in neig_areas:
    if neig_area not in final_neig_areas:
        final_neig_areas.append(neig_area)


In [14]:
def determine_perimeter(region, label):
    count = 0
    for r in region:
        if input[r[0]-1][r[1]] != label:
            count += 1
        if input[r[0]+1][r[1]] != label:
            count += 1
        if input[r[0]][r[1]-1] != label:
            count += 1
        if input[r[0]][r[1]+1] != label:
            count += 1
    return count

tot = 0

for neig in final_neig_areas:
    label = neig['label']
    area = len(neig['pos'])
    per_len = determine_perimeter(neig['pos'], label)
    tot += area * per_len
print(tot)


1304764


In [4]:
def search_for_x(null_map,label):
    found_x = 0
    for r in range(len(null_map)-1):
        for p in range(len(null_map[0])-1):
            if null_map[r][p] == label and null_map[r][p+1] == 0 and null_map[r+1][p] == 0 and null_map[r+1][p+1] == label :
                found_x += 1
            if null_map[r][p] == 0 and null_map[r][p+1] == label and null_map[r+1][p] == label and null_map[r+1][p+1] == 0:
                found_x += 1
    return found_x


In [5]:
import copy
def get_angles(i):
    null_map = copy.deepcopy(input)
    count = 0
    figure = final_neig_areas[i]['pos']
    label = final_neig_areas[i]['label']
    
    # set all values in null_map to 0 if they are not in the figure
    for r in range(len(null_map)):
        for p in range(len(null_map[r])):
            if (r,p) not in figure:
                null_map[r][p] = 0

    area = len(figure)
    for el in figure:
        angle_l_up = {"up":(null_map[el[0]-1][el[1]]), "left":(null_map[el[0]][el[1]-1]), "diag":(null_map[el[0]-1][el[1]-1])}
        angle_l_down = {"down":(null_map[el[0]+1][el[1]]), "left":(null_map[el[0]][el[1]-1]), "diag":(null_map[el[0]+1][el[1]-1])}
        angle_r_up = {"up":(null_map[el[0]-1][el[1]]), "right":(null_map[el[0]][el[1]+1]), "diag":(null_map[el[0]-1][el[1]+1])}
        angle_r_down = {"down":(null_map[el[0]+1][el[1]]), "right":(null_map[el[0]][el[1]+1]), "diag":(null_map[el[0]+1][el[1]+1])}
        if label not in [angle_l_up["up"], angle_l_up["left"], angle_l_up["diag"]]:
            count+=1
        if label not in [angle_l_down["down"], angle_l_down["left"], angle_l_down["diag"]]:
            count+=1
        if label not in [angle_r_up["up"], angle_r_up["right"], angle_r_up["diag"]]:
            count+=1
        if label not in [angle_r_down["down"], angle_r_down["right"], angle_r_down["diag"]]:
            count+=1
        # if two out of three are label count+=1
        if [angle_l_up["up"], angle_l_up["left"], angle_l_up["diag"]].count(label) == 2 and  angle_l_up["diag"] != label:
            count+=1
        if [angle_l_down["down"], angle_l_down["left"], angle_l_down["diag"]].count(label) == 2 and angle_l_down["diag"] != label:
            count+=1
        if [angle_r_up["up"], angle_r_up["right"], angle_r_up["diag"]].count(label) == 2 and angle_r_up["diag"] != label:
            count+=1
        if [angle_r_down["down"], angle_r_down["right"], angle_r_down["diag"]].count(label) == 2 and angle_r_down["diag"] != label:
            count+=1
    check = search_for_x(null_map,label)
    for j in range(check):
        count+=2
    number = count*area
    return number

In [6]:
tot = 0
for i in range(len(final_neig_areas)):
    tot += get_angles(i)
print (tot)

811148
